<a href="https://colab.research.google.com/github/arize-ai/phoenix/blob/main/tutorials/experiments/run_experiments_with_llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!uv pip install arize-phoenix "torch<2.7" sentence-transformers openinference-instrumentation-llama_index openinference-instrumentation-openai llama-index

In [ ]:
import json
import os
from getpass import getpass
from typing import Any

import pandas as pd

from phoenix.evals import (
    OpenAIModel,
)

pd.set_option("display.max_colwidth", None)

In [ ]:
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")

In [ ]:
import tempfile
from datetime import datetime, timezone
from functools import partial
from time import sleep
from urllib.request import urlretrieve

import nest_asyncio
import pandas as pd
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.settings import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from openinference.instrumentation.openai import OpenAIInstrumentor

import phoenix as px
from phoenix.client import Client
from phoenix.experiments.evaluators import ConcisenessEvaluator
from phoenix.otel import register

nest_asyncio.apply()

# Instrument LlamaIndex

In [ ]:
tracer_provider = register(endpoint="http://127.0.0.1:6006/v1/traces")
LlamaIndexInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)
OpenAIInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

# Create Dataset

In [ ]:
df = pd.DataFrame(
    {
        "input_messages": [
            [{"role": "user", "content": "Which grad schools did the author apply for and why?"}],
            [{"role": "user", "content": "What did the author do growing up?"}],
        ],
        "output_message": [
            {
                "role": "assistant",
                "content": "The author applied to three grad schools: MIT and Yale, which were renowned for AI at the time, and Harvard, which the author had visited because a friend went there and it was also home to Bill Woods, who had invented the type of parser the author used in his SHRDLU clone. The author chose these schools because he wanted to learn about AI and Lisp, and these schools were known for their expertise in these areas.",
            },
            {
                "role": "assistant",
                "content": "The author took a painting class at Harvard with Idelle Weber and later became her de facto studio assistant. Additionally, the author worked on several different projects, including writing essays, developing spam filters, and painting.",
            },
        ],
    }
)
for c in ("input_messages", "output_message"):
    df[c] = df[c].apply(json.dumps).astype("string")
df

## Upload Dataset

In [ ]:
px.launch_app()

In [ ]:
dataset_name = datetime.now(timezone.utc).isoformat()
phoenix_client = Client(base_url="http://localhost:6006")

phoenix_client.datasets.create_dataset(
    name=dataset_name,
    dataframe=df,
    input_keys=("input_messages",),
    output_keys=("output_message",),
)
sleep(1)

## Download Dataset

In [ ]:
ds = phoenix_client.datasets.get_dataset(dataset=dataset_name)

# Set Up Experiment Metadata

In [ ]:
experiment_metadata = {
    "llm": "gpt-4",
    "embed_model": "text-embedding-3-small",
    "reranker": "cross-encoder/ms-marco-MiniLM-L-2-v2",
}

# Set Up LLamaIndex

In [ ]:
Settings.llm = OpenAI(model=experiment_metadata["llm"])
Settings.embed_model = OpenAIEmbedding(model=experiment_metadata["embed_model"])
reranker = SentenceTransformerRerank(model=experiment_metadata["reranker"], top_n=2)

essay = "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt"
with tempfile.NamedTemporaryFile() as tf:
    urlretrieve(essay, tf.name)
    documents = SimpleDirectoryReader(input_files=[tf.name]).load_data()
index = VectorStoreIndex.from_documents(documents)

# Set Up Capture of Retrieved Documents

# Create Task

In [ ]:
def rag_with_reranker(input) -> str:
    chat_engine = index.as_chat_engine(similarity_top_k=10, node_postprocessors=[reranker])
    messages = input["input_messages"]
    messages = json.loads(messages)
    response = chat_engine.chat(messages[-1]["content"])
    return str(response)

# Define Evaluators

In [ ]:
def contains_substring(output, substring: str) -> dict[str, Any]:
    score = int(isinstance(output, str) and substring in output)
    return {"score": score, "explanation": f"the substring `{substring}` was in the output"}

In [ ]:
print(ConcisenessEvaluator.template)

# Run Experiment with Evaluators

In [ ]:
model = OpenAIModel(model="gpt-4o")

experiment = phoenix_client.experiments.run_experiment(
    dataset=ds,
    task=rag_with_reranker,
    experiment_metadata=experiment_metadata,
    evaluators=[partial(contains_substring, substring="school"), ConcisenessEvaluator(model)],
)